Search engine
====

The dataset
-----

In this exercise we use a wikipedia *Star Wars* dataset. 
This is a small dataset of wikipedia abstracts on *Star Wars* related topics. 

The dataset is a couple $\langle L,Q \rangle$:
- $L:I \mapsto D$ is a library function. Each document has a unique index $i\in I$ that maps to the document string $d\in D$.
- $Q = (q_1,i_1) \ldots (q_n,i_n)$  is a list of reference (queries,index) couples.
  Each mapping a query string to the index of a page

We further have a set of links ${\cal L} \subseteq I\times I$ between the document indexes. 
These links are encoded as a graph whose vertices are indexes and edges are couples of indexes. 



In [40]:
import wikipedia
import networkx as nx
import random


# This code gathers wikipedia pages and creates the dataset from these pages.
# The queries are the page titles, the documents are paragraphs randomly selected in the pages
# We further create a graph of links between document ids expressed as an adjacency matrix

pages = [
           "Luke Skywalker","Admiral Ackbar","Boba Fett","C-3PO",
           "Chewbacca","Count Dooku","Darth Maul","Darth Vader","General Grievous",
           "Greedo","Han Solo","Jabba the Hutt","Jango Fett","Jar Jar Binks","Kylo Ren",
           "Lando Calrissian","Leia Organa","Mace Windu","The Mandalorian (character)","Obi-Wan Kenobi",
           "Padmé Amidala","Palpatine","Poe Dameron","Supreme Leader Snoke","Qui-Gon Jinn","R2-D2",
           "Rey (Star Wars)","Saw Gerrera","Wedge Antilles","Yoda"
          ]


# Gets the wikipedia pages
def build_library(nodeset,k):
    """
    Builds and indexes the documents and queries from wikipedia pages
    
    Args: 
       nodeset (list): a list of strings, titles of wikipedia pages
       k        (int): the number of paragraphs to sample for some wikipedia page
    Returns:
        a dict of documents idx:str , 
        a list of query:idx. A single query may have multiple idx in the list         
    """
    random.seed(77)
    nodes2pages = {node:wikipedia.page(node,auto_suggest=False) for node in nodeset}

    idx       = 1
    documents = {}
    queries   = []
    
    for node in nodeset:
        cpage      = nodes2pages[node]
        query      = cpage.title
        paragraphs = [line for line in cpage.content.split('\n') if len(line) > 140 and not line.startswith('=') ]
        paragraphs = random.sample(paragraphs,k)
        
        for parag in paragraphs:
             documents[idx] = parag
             queries.append( (query,idx) )
             idx += 1
    return documents, queries


def link_documents(library,queries):
    """
    Creates a web structure for a set of documents.
    Adds a link from i to j if the content of i contains a mention of j
    
    Args:
        library (dict): a dict of idx:documents
        queries (list): a list of (query,idx)
    Returns:
        a list of document ids, a networkx DiGraph
    """
    docids = [docidx for docidx in library] 
    
    graph  = nx.DiGraph()
    graph.add_nodes_from(docids)

    idx2queries = {idx:q for q,idx in queries}
    for docidx, doc in library.items():
        for qidx,title in idx2queries.items():
            if title.lower() in doc.lower():
                graph.add_edge(docidx,qidx)

    return docids,nx.to_numpy_array(graph)



documents,queries = build_library(pages,10)
docids, adjacency = link_documents(documents,queries)

PageRank
----

In this section we want to implement a PageRank algorithm for the Star Wars documents.
We already have the adjacency matrix. The page rank function outputs a dictionary,
or the function $PR:I\mapsto \mathbb{R}$ where each document index is mapped to its PageRank score.

In [37]:
import numpy as np
from numpy.linalg import norm

def pagerank(docids,adjacency,alpha=0.85,eps=0.0001):
    """
    Computes the PageRank of a dense adjacency matrix
    Args:
       docids    (list)    : a list of document ids of len equal to the adjacency size
       adjacency (np.array): an adjacency matrix
       alpha        (float): the alpha for mixing when creating the Google Matrix.  
       eps          (float): epsilon for stopping PageRank iterations
    Returns: 
        dict. A dictionary whose keys are document ids and values are document page ranks
    """
    assert len(docids) == len(adjacency) 
    ### BEGIN SOLUTION
    n,_ = adjacency.shape

    with np.errstate(divide='ignore',invalid='ignore'):
      row_stochastic   = adjacency / adjacency.sum(axis=1,keepdims=True)
      row_stochastic   = np.nan_to_num(row_stochastic,0.)

    jump             = np.ones_like(row_stochastic) / n

    google           = (alpha * row_stochastic + (1-alpha) * jump )
    #ensure stochasticity for absorbing nodes
    google           = google / google.sum(axis=1,keepdims=True) 
    #google matrix is eventually column stochastic
    google = google.T


    x   = np.random.random(n)
    x  /= x.sum() 	                   #ensure probabilistic vector
    error = float('inf')
    idx = 0
    while error >= eps:
        xnext = google @ x
        error = norm(xnext-x)
        print(f'Iteration {idx}, Error = {error}')
        x     = xnext
        idx  += 1
        
    return dict(zip(docids,x))
	
    ### END SOLUTION

pr = pagerank(docids,adjacency)

m = 0
for docid,score in sorted(pr.items(),key=lambda x:x[1],reverse=True):
    print(documents[docid],score)
    print()


Iteration 0, Error = 0.04191552999472353
Iteration 1, Error = 0.008299958993177167
Iteration 2, Error = 0.003639073079624897
Iteration 3, Error = 0.0018633296722965606
Iteration 4, Error = 0.0010272470956698577
Iteration 5, Error = 0.0005850886396207448
Iteration 6, Error = 0.0003405463954349739
Iteration 7, Error = 0.00020210924254983892
Iteration 8, Error = 0.00012226446415435625
Iteration 9, Error = 7.53137803105765e-05
In 1980, American musician and parody artist "Weird Al" Yankovic wrote and recorded a parody of the Kinks' "Lola", called "Yoda". It was later rerecorded for his 1985 album Dare to Be Stupid, after Yankovic was able to obtain permission from both George Lucas and the Kinks. 0.009718055266606728

In The Force Awakens, set 30 years after Yoda's death in Return of the Jedi, Yoda's voice is heard by the young scavenger Rey in a Force vision after she discovers Luke Skywalker's lightsaber under a castle owned by Maz Kanata. 0.009718055266606728

Yoda returns as a younger 

Document vectorization
-------

Vector space based retrieval
----

The ranking problem
----

We currently have two ranking methods:
- PageRank, $PR(d)$, that scores documents wrt to their hyperlinks
- The vector space model, $VSM(q,d)$, that scores documents wrt to their similarity with the query.




In this exercise, we propose to rank the search results according to the scores given by the formula:

$$
r(q,d) = \lambda\, VSM(q,d)  + (1-\lambda)\, PR(d)
$$

where $\lambda \in [0,1]$ is a weight. When set to 1 $PR$ is ignored and when set to 0 $VSM$ is ignored.
